In [1]:
#####training q_resnet18
import sys
import time
import random
import os
sys.path.append("..")
from module.layer_fusion_module import *
from module.resnet_module import *
random_seed = 0
num_classes = 10
cuda_device = torch.device("cuda:0")
cpu_device = torch.device("cpu:0")

model_dir = "saved_models"
model_filename = "q_resnet18_cifar10.pt"
quantized_model_filename = "q_resnet18_quantized_cifar10.pt"
model_filepath = os.path.join(model_dir, model_filename)
quantized_model_filepath = os.path.join(model_dir, quantized_model_filename)

set_random_seeds(random_seed=random_seed)
    
    # Create an untrained model.
model = create_model(num_classes=num_classes)

train_loader, test_loader = prepare_dataloader(num_workers=0, train_batch_size=128, eval_batch_size=256)
    
# Train model.
print("Training Model...")
model = train_model(model=model, train_loader=train_loader, test_loader=test_loader, device=cuda_device, learning_rate=1e-1, num_epochs=10)
# Save model.
save_model(model=model, model_dir=model_dir, model_filename=model_filename)
    
# ① floating point 타입으로 모델을 학습하거나 pre-trained 모델을 불러옵니다.
# Load a pretrained model.
model = load_model(model=model, model_filepath=model_filepath, device=cuda_device)
# Move the model to CPU since static quantization does not support CUDA currently.
    
# ② 모델을 CPU 상태로 두고 학습 모드로 변환합니다. (model.train())
model.to(cpu_device)
# Make a copy of the model for layer fusion
fused_model = copy.deepcopy(model)

model.train()
# The model has to be switched to training mode before any layer fusion.
# Otherwise the quantization aware training will not work correctly.
fused_model.train()

Files already downloaded and verified
Files already downloaded and verified
Training Model...
Epoch: -1 Eval Loss: 2.312 Eval Acc: 0.107
Epoch: 000 Train Loss: 3.069 Train Acc: 0.199 Eval Loss: 1.919 Eval Acc: 0.272
Epoch: 001 Train Loss: 1.748 Train Acc: 0.331 Eval Loss: 1.598 Eval Acc: 0.393
Epoch: 002 Train Loss: 1.582 Train Acc: 0.403 Eval Loss: 1.555 Eval Acc: 0.421
Epoch: 003 Train Loss: 1.462 Train Acc: 0.456 Eval Loss: 1.383 Eval Acc: 0.493
Epoch: 004 Train Loss: 1.365 Train Acc: 0.497 Eval Loss: 1.318 Eval Acc: 0.528
Epoch: 005 Train Loss: 1.283 Train Acc: 0.532 Eval Loss: 1.266 Eval Acc: 0.540
Epoch: 006 Train Loss: 1.194 Train Acc: 0.568 Eval Loss: 1.317 Eval Acc: 0.542
Epoch: 007 Train Loss: 1.127 Train Acc: 0.596 Eval Loss: 1.133 Eval Acc: 0.604
Epoch: 008 Train Loss: 1.072 Train Acc: 0.615 Eval Loss: 1.042 Eval Acc: 0.639
Epoch: 009 Train Loss: 1.020 Train Acc: 0.637 Eval Loss: 1.206 Eval Acc: 0.641


ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (conv2_x): Sequential(
    (0): q_BasicBlock(
      (residual_function): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
      )
      (shortcut): Sequential()
      (q_func): FloatFunctional(
        (activation_post_process): Identity()
      )
      (relu): ReLU()
    )
    (1): q_BasicBlock(
      (residual_fun

In [2]:
#QAT
def fuse_model(model,model_name):
    fusion_layer_dict= make_fuse_dict(model,model_name)
    for key in fusion_layer_dict:
        for fuse_group in fusion_layer_dict[key]:        
            torch.quantization.fuse_modules(eval(key), [fuse_group], inplace=True) 
model_dir = "saved_models"
model_filename = "q_resnet18_cifar10.pt"
fusioned_model_filename = "q_fusioned_resnet18_cifar10.pt"
quantized_model_filename_jit = "q_resnet18_quantized_cifar10_jit.pt"
quantized_model_filename = "q_resnet18_quantized_cifar10.pt"
model_filepath = os.path.join(model_dir, model_filename)
quantized_model_filepath = os.path.join(model_dir, quantized_model_filename)
quantized_model_filepath_jit = os.path.join(model_dir, quantized_model_filename_jit)

quantized_model = QuantizedResNet18(model_fp32=fused_model)

quantization_config = torch.quantization.get_default_qconfig("fbgemm")

quantized_model.qconfig = quantization_config
fuse_model(quantized_model,'quantized_model')
torch.quantization.prepare_qat(quantized_model, inplace=True)

print("Training QAT Model...")
quantized_model.train()

train_model(model=quantized_model, train_loader=train_loader, test_loader=test_loader, device=cuda_device, learning_rate=1e-3, num_epochs=10)
quantized_model.to(cpu_device)    
quantized_model = torch.quantization.convert(quantized_model, inplace=True)

quantized_model.eval()
save_torchscript_model(model=quantized_model, model_dir=model_dir, model_filename=quantized_model_filename_jit)
save_model(model=quantized_model, model_dir=model_dir, model_filename=quantized_model_filename)


C:\Users\win0\Anaconda3\envs\pytorch\lib\site-packages\torch\ao\quantization\observer.py:174: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  reduce_range will be deprecated in a future release of PyTorch."


Training QAT Model...
Epoch: -1 Eval Loss: 1.206 Eval Acc: 0.641
Epoch: 000 Train Loss: 0.923 Train Acc: 0.672 Eval Loss: 0.910 Eval Acc: 0.690
Epoch: 001 Train Loss: 0.894 Train Acc: 0.683 Eval Loss: 0.932 Eval Acc: 0.691
Epoch: 002 Train Loss: 0.883 Train Acc: 0.686 Eval Loss: 0.946 Eval Acc: 0.693
Epoch: 003 Train Loss: 0.872 Train Acc: 0.690 Eval Loss: 0.932 Eval Acc: 0.695
Epoch: 004 Train Loss: 0.870 Train Acc: 0.691 Eval Loss: 0.979 Eval Acc: 0.696
Epoch: 005 Train Loss: 0.865 Train Acc: 0.691 Eval Loss: 0.915 Eval Acc: 0.699
Epoch: 006 Train Loss: 0.858 Train Acc: 0.695 Eval Loss: 0.913 Eval Acc: 0.699
Epoch: 007 Train Loss: 0.852 Train Acc: 0.697 Eval Loss: 0.943 Eval Acc: 0.699
Epoch: 008 Train Loss: 0.853 Train Acc: 0.695 Eval Loss: 0.954 Eval Acc: 0.700
Epoch: 009 Train Loss: 0.846 Train Acc: 0.698 Eval Loss: 0.884 Eval Acc: 0.706


C:\Users\win0\Anaconda3\envs\pytorch\lib\site-packages\torch\ao\quantization\observer.py:886: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  src_bin_begin // dst_bin_width, 0, self.dst_nbins - 1
C:\Users\win0\Anaconda3\envs\pytorch\lib\site-packages\torch\ao\quantization\observer.py:891: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  sr

In [3]:
# Load quantized model_jit.
quantized_jit_model = load_torchscript_model(model_filepath=quantized_model_filepath_jit, device=cpu_device)
_, fp32_eval_accuracy = evaluate_model(model=model, test_loader=test_loader, device=cuda_device, criterion=None)
_, int8_eval_accuracy = evaluate_model(model=quantized_jit_model, test_loader=test_loader, device=cpu_device, criterion=None)
#_, int8_eval_accuracy = evaluate_model(model=quantized_model, test_loader=test_loader, device=cpu_device, criterion=None)
print("FP32 evaluation accuracy: {:.3f}".format(fp32_eval_accuracy))
print("INT8 evaluation accuracy: {:.3f}".format(int8_eval_accuracy))

fp32_cpu_inference_latency = measure_inference_latency(model=model, device=cpu_device, input_size=(1,3,32,32), num_samples=100)
int8_cpu_inference_latency = measure_inference_latency(model=quantized_model, device=cpu_device, input_size=(1,3,32,32), num_samples=100)
int8_jit_cpu_inference_latency = measure_inference_latency(model=quantized_jit_model, device=cpu_device, input_size=(1,3,32,32), num_samples=100)
fp32_gpu_inference_latency = measure_inference_latency(model=model, device=cuda_device, input_size=(1,3,32,32), num_samples=100)

print("FP32 CPU Inference Latency: {:.2f} ms / sample".format(fp32_cpu_inference_latency * 1000))
print("FP32 CUDA Inference Latency: {:.2f} ms / sample".format(fp32_gpu_inference_latency * 1000))
print("INT8 CPU Inference Latency: {:.2f} ms / sample".format(int8_cpu_inference_latency * 1000))
print("INT8 JIT CPU Inference Latency: {:.2f} ms / sample".format(int8_jit_cpu_inference_latency * 1000))

FP32 evaluation accuracy: 0.641
INT8 evaluation accuracy: 0.682
FP32 CPU Inference Latency: 3.28 ms / sample
FP32 CUDA Inference Latency: 4.40 ms / sample
INT8 CPU Inference Latency: 2.20 ms / sample
INT8 JIT CPU Inference Latency: 0.65 ms / sample
